# Computability, Complexity, and Languages, Second Edition Martin Davis


In [367]:
import re

# for encoding:
$$ \langle x,y\rangle=z : z = max(2^x(2y+1) - 1,0) $$
# for decoding:
$$ \langle x,y\rangle = z : (2y+1) = \frac {z+1}{x^2} $$

In [368]:
def encode(x,y):
    return max(2**x*(2*y+1)-1,0 )

def decode(z):
    z+=1
    x = 0
    while not(z%2):
        z=z//2
        x+=1
    y = (z-1)//2
    return x,y

In [369]:
def getvar(inp):
    '''this function
    if inp is str return number Varable(int)
    else if inp is int return Varable(str)
    else raise TypeError'''
    
    var = ("Y","X","Z")
    
    if type(inp) is str:
        s=inp
        
        alpha,num = re.match(r"([\D]+)([\d]*)",s).groups()
        if num=="":
            num=1
        if alpha not in var:
            raise Exception("Variable not found")
        v = var.index(alpha)
        n = int(num)
        return (n-1)*2+v
    elif type(inp) is int:
        n=inp
        
        if n == 0:
            return var[0]
        elif n%2==1:
            return f"{var[1]}{(n+1)//2}"
        elif n%2==0:
            return f"{var[2]}{(n+1)//2}"
    raise TypeError    

In [370]:
def getlabel(inp):
    '''this function
    if inp is str return number label(int)
    else if inp is int return label(str)
    else raise TypeError'''
    label = ("A","B","C","D","E")
    
    if type(inp) is str:
        s=inp

        alpha,num = re.match(r"([\D]+)([\d]*)",s).groups()
        if num=="":
            num = 1
        if alpha not in label:
            raise Exception("Label not found")
        l = label.index(alpha)
        n = int(num)
        return (n-1)*5+l+1
    
    elif type(inp) is int:
        n=inp
        if n == 0:
            return ""
        l = label[(n-1)%5]
        num = ((n-1)//5)+1
        if num==1:
            num = ""
        return f"{l}{num}"
    
    raise TypeError

In [371]:
def convert_to_num(s:str)->tuple:
    """This function receives the program instruction and converts it to a number:
    args:
        s is instruction 
    return
        <a,<b,c>>, #P"""
    s1 = re.sub(r"\s","",s)
    s1 = re.sub(r"\]","] ",s1)
    b0 = re.match(r"\[?([\w]*)\]?[\s]?([\w]*)<-[\w]+",s1)
    b1 = re.match(r"\[?([\w]*)\]?[\s]?([\w]*)<-[\w]+\+1",s1)
    b2 = re.match(r"\[?([\w]*)\]?[\s]?([\w]*)<-[\w]+\-1",s1)
    b3 = re.match(r'\[?([\w]*)\]?[\s]?IF([\w]*)!=0GOTO([\w]*)',s1)
    ll = ""
    if b1:
        b= 1
        l,v  = b1.groups()
        if v=="":
            l,v = "",l
    elif b2:
        b= 2
        l,v  = b2.groups()
        if v=="":
            l,v = "",l
    elif b0:
        b= 0
        l,v  = b0.groups()
        if v=="":
            l,v = "",l
    elif b3:
        l,v,ll=b3.groups()
        b = getlabel(ll)+2
    
#     print(s,b,l,v,ll,sep=",")
    if l=="":
        a = 0
    else:
        a = getlabel(l)
    c= getvar(v)
    num = encode(a,encode(b,c))
    print(f"{s:25}<{a},<{b},{c}>>:{num:>5}")
    
    return (a,b,c) , num

In [372]:
def convert_to_program(num:int)->str:
    """This function receives the number and converts it to program instruction:
    args:
        num is number of program instruction
    return
        instruction"""
    a,bc = decode(num)
    b,c = decode(bc)
    if a == 0 :
        s = "    "
    else:
        s = f"[{getlabel(a)}] "

    v = getvar(c)
    if b==0:
        s+=f"{v}<-{v}"
    elif b==1:
        s+=f"{v}<-{v}+1"
    elif b==2:
        s+=f"{v}<-{v}-1"
    else:
        ll = getlabel(b-2)
        s+=f"IF {v}!=0 GOTO {ll}"
    print(f"<{a},<{b},{c}>>:{num:>5}\t{s}")
    return s


# Convert Program to number (Godel Number)

In [373]:
program = """[A] X<-X+1
    IF X!=0 GOTO A"""

In [374]:
m = []
for s in program.split("\n"):
    m.append(convert_to_num(s)[1])
    


[A] X<-X+1               <1,<1,1>>:   21
    IF X!=0 GOTO A       <0,<3,1>>:   46


In [375]:
print(m)

[21, 46]


# convert number (Godel Number) to Program

In [376]:
m2 = [21, 46]

In [377]:
program2 = []
for num in m2:
    program2.append(convert_to_program(num))
    


<1,<1,1>>:   21	[A] X1<-X1+1
<0,<3,1>>:   46	    IF X1!=0 GOTO A


In [378]:
print('\n'.join(program2))

[A] X1<-X1+1
    IF X1!=0 GOTO A
